# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url	slides_url
2009-10-01	Paper Title Number 1	Journal 1	This paper is about the number 1. The number 2 is left for future work.	Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).	paper-title-number-1	http://academicpages.github.io/files/paper1.pdf	http://academicpages.github.io/files/slides1.pdf
2010-10-01	Paper Title Number 2	Journal 1	This paper is about the number 2. The number 3 is left for future work.	Your Name, You. (2010). "Paper Title Number 2." <i>Journal 1</i>. 1(2).	paper-title-number-2	http://academicpages.github.io/files/paper2.pdf	http://academicpages.github.io/files/slides2.pdf
2015-10-01	Paper Title Number 3	Journal 1	This paper is about the number 3. The number 4 is left for future work.	Your Name, You. (2015). "Paper Title Number 3." <i>Journal 1</i>. 1(3).	paper-title-number-3	http://academicpages.github.io/files/paper3.pdf	http://academicpages.github.io/files/slides3.pdf

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [4]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,2022-06-01,Planar potential flow on Cartesian grids,Journal of Fluid Mechanics,NaN,"Beckers, D., Eldredge, J.D. (2022). ""Planar po...",planar-potential-flow-on-cartesian-grids,https://www.cambridge.org/core/product/identif...,NaN
1,2026-01-01,Improved accuracy of the discrete immersed bou...,Under review,NaN,"Beckers, D., Bae, H.J., Goza, A. (2026). ""Impr...",improved-accuracy-of-the-discrete-immersed-bou...,https://baeresearchgroup.slack.com/archives/D0...,NaN
2,2025-10-01,Model-Based Reinforcement Learning for Control...,AIAA Journal,NaN,"Liu, Z., Beckers, D., Eldredge, J.D. (2025). ""...",model-based-reinforcement-learning-for-control...,https://doi.org/10.2514/1.J064790,NaN
3,2024-09-01,Deep reinforcement learning of airfoil pitch c...,Phys. Rev. Fluids,NaN,"Beckers, D., Eldredge, J.D. (2024). ""Deep rein...",deep-reinforcement-learning-of-airfoil-pitch-c...,https://link.aps.org/doi/10.1103/PhysRevFluids...,NaN
4,2023-06-01,Wind tunnel effects on gust-interaction simula...,Theor. Comput. Fluid Dyn.,NaN,"Beckers, D., Eldredge, J.D. (2023). ""Wind tunn...",wind-tunnel-effects-on-gust-interaction-simula...,https://doi.org/10.1007/s00162-023-00668-9,NaN
5,2026-01-01,A High-Fidelity Simulation Framework for Turbu...,AIAA SCITECH 2026 Forum,NaN,"Beckers, D., Balasubramanian, S., Lin, C.-T., ...",a-high-fidelity-simulation-framework-for-turbu...,NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [5]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [6]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [8]:
!ls ../_publications/

2022-06-01-planar-potential-flow-on-cartesian-grids.md
2023-06-01-wind-tunnel-effects-on-gust-interaction-simulations.md
2024-09-01-deep-reinforcement-learning-of-airfoil-pitch-control-in-a-highly-disturbed-environment.md
2025-10-01-model-based-reinforcement-learning-for-control-of-strongly-disturbed-unsteady-aerodynamic-flows.md
2026-01-01-a-high-fidelity-simulation-framework-for-turbulent-flows-with-complex-metamaterial-structures.md
2026-01-01-improved-accuracy-of-the-discrete-immersed-boundary-formulation.md


In [12]:
!cat ../_publications/2022-06-01-planar-potential-flow-on-cartesian-grids.md

---
title: "Planar potential flow on Cartesian grids"
collection: publications
permalink: /publication/2022-06-01-planar-potential-flow-on-cartesian-grids
date: 2022-06-01
venue: 'Journal of Fluid Mechanics'
paperurl: 'https://www.cambridge.org/core/product/identifier/S0022112022002385/type/journal_article'
citation: 'Beckers, D., Eldredge, J.D. (2022). &quot;Planar potential flow on Cartesian grids.&quot; <i>Journal of Fluid Mechanics</i>. 941: A19.'
---
[Download paper here](https://www.cambridge.org/core/product/identifier/S0022112022002385/type/journal_article)

Recommended citation: Beckers, D., Eldredge, J.D. (2022). "Planar potential flow on Cartesian grids." <i>Journal of Fluid Mechanics</i>. 941: A19.